In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import geopandas as gpd
import pandas as pd


scripts_dir = Path("../..").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())
from helpers.mapshaper import Mapshaper
from helpers.tippcanoe import mbtileGeneration
from helpers.mapbox_uploader import uploadToMapbox
from helpers.settings import get_settings
from helpers.file_handler import FileConventionHandler
from helpers.utils import download_and_unzip_if_needed

from data_commons.loader import load_regions

In [3]:
mysettings = get_settings()
prev_step = "preprocess"
current_step = "tiles"

### EEZs: Exclusive Economic Zones 

In [5]:
pipe = "eez"
collection_name = f"{pipe}_v11"

eez_dir = FileConventionHandler(pipe)
# Download the EEZ file && unzip it
download_and_unzip_if_needed(eez_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(8).input([eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields=",".join(["GEONAME", "POL_TYPE", "ISO_SOV1", "ISO_SOV2", "ISO_SOV3"])
).output(
    eez_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()
mbtileGeneration(
    eez_dir.get_step_fmt_file_path(current_step, "json"),
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
)

/home/mambauser/data/eez/processed/eez_preprocess.zip
/home/mambauser/data/eez/processed/preprocess


Allocating 8 GB of heap memory
[o] Wrote /home/mambauser/data/eez/processed/tiles/eez_tiles.json


PosixPath('/home/mambauser/data/eez/processed/tiles/eez_v11.mbtiles')

In [ ]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

#### EEZs: wdpa Regions

In [29]:
collection_name = "regions"

# load the EEZ file & the regions file
eez_data = gpd.read_file(eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix())
regions_df = pd.DataFrame(
    [
        {"region_id": data["region_iso"], "location_id": iso}
        for data in load_regions().get("data", [])
        for iso in data["country_iso_3s"]
    ]
)

# merge the two files
gpd.GeoDataFrame(
    pd.merge(
        eez_data,
        regions_df,
        how="left",
        left_on="ISO_SOV1",
        right_on="location_id",
        sort=True,
        copy=True,
    ),
    crs=eez_data.crs,
).to_file(
    filename=eez_dir.get_processed_step_path(prev_step)
    .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
    .as_posix(),
    driver="ESRI Shapefile",
)

# dissolve by region_id

Mapshaper(16).input(
    [
        eez_dir.get_processed_step_path(prev_step)
        .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
        .as_posix()
    ]
).dissolve2(fields="region_id").output(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json").as_posix(),
    force=True,
    format="geojson",
).execute()

# generate the mbtiles
mbtileGeneration(eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json"))

In [34]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/eez/processed/tiles/regions.mbtiles to s3://tilestream-tilesets-production/97/_pending/ojc7oxn5cpu10yo0o9tsl1xlc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:00<00:00,  1.81s/it]


True

### MPAs: Marine Protected Areas from WDPA

In [35]:
pipe = "mpa"
collection_name = "mpas_wdpa"

source_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(16).input([source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields="WDPAID,NAME,PA_DEF,GIS_M_AREA,PARENT_ISO"
).clean(allow_overlaps=True, rewind=True).simplify("dp 10% keep-shapes planar").clean(
    allow_overlaps=True
).output(
    source_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# generate the mbtiles
mbtileGeneration(source_dir.get_step_fmt_file_path(current_step, "json"))

In [ ]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

### MPA Atlas 

In [45]:
pipe = "mpaatlas"
collection_name = "mpa_atlas"

source_dir = FileConventionHandler(pipe)
# Download the data file && unzip it if needed
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# generate the mbtiles
mbtileGeneration(
    source_dir.get_step_fmt_file_path(prev_step, "shp"),
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
)

In [ ]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

### Protected seas

In [50]:
pipe = "protectedseas"
collection_name = "protected_seas"

source_dir = FileConventionHandler(pipe)

# Download the data file && unzip it if needed
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(16).input([source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).clean(
    allow_overlaps=True, rewind=True
).simplify("dp 10% keep-shapes planar").clean(allow_overlaps=True).output(
    source_dir.get_step_fmt_file_path(current_step, "json").as_posix(),
    force=True,
    format="geojson",
).execute()

# generate the mbtiles
mbtileGeneration(source_dir.get_step_fmt_file_path(current_step, "json"))

/home/mambauser/data/protectedseas/processed/protectedseas_preprocess.zip
/home/mambauser/data/protectedseas/processed/preprocess


Allocating 16 GB of heap memory
[clean] Retained 10,879 of 10,879 features
[simplify] Repaired 110,498 intersections; 22,095 intersections could not be repaired
[clean] Retained 10,879 of 10,879 features
[o] Wrote /home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.json
For layer 0, using name "protectedseas_tiles"
/home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.json:967: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.json:378: Found ] at top level
10879 features, 31017199 bytes of geometry, 377988 bytes of separate metadata, 766466 bytes of string pool
Choosing a maxzoom of -z3 for features about 58402 feet (17801 meters) apart
Choosing a maxzoom of -z11 for resolution of about 245 feet (74 meters) within features
  99.9%  11/673/930    


PosixPath('/home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.mbtiles')

In [ ]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    "protected_seas",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

### Habitat layers

In [ ]:
# TODO: Add all the habitats layer processing
habitat = mysettings.DATA_DIR.joinpath("habitat_intermediate", "layer").resolve()

#### Warm water corals

In [ ]:
warm_water_corals = habitat.joinpath("warm_water_corals.mbtiles").resolve()

In [ ]:
uploadToMapbox(
    warm_water_corals,
    "warm_water_corals",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

### Contextual layers

In [ ]:
# Todo: we need to add the contextual layers upload here